In [1]:
from shapely.geometry import box
import matplotlib.pyplot as plt
from contextily import add_basemap
import contextily as cx
from shapely.geometry import Point, Polygon
from shapely.ops import transform
from pyproj import Transformer
import geopandas as gpd
import numpy as np
import time
import requests
import pandas as pd

In [2]:
import geopandas as gpd
from shapely.geometry import box
import matplotlib.pyplot as plt
from contextily import add_basemap
import contextily as cx

In [5]:
# 选择适合的投影坐标系统
projected_crs = '3857'

parks_detail = gpd.read_file('park_details_isochrones_10_walking_gdf.geojson')
parks_detail = parks_detail.to_crs(epsg = projected_crs)
parks_detail.columns

Index(['OBJECTID', 'NAME', 'PMA_NAME', 'ADDRESS', 'PIN', 'SUBPARCEL',
       'TOTAL_AREA', 'OWNER', 'LEASE', 'MAINT', 'TYPE', 'ACQ_DATE', 'NAMEFLAG',
       'REVIEW_DATE', 'AMWOID', 'SDQL', 'SE_ANNO_CAD_DATA', 'GIS_CRT_DT',
       'GIS_EDT_DT', 'GlobalID', 'centroid', 'isochrones', 'geometry'],
      dtype='object')

In [8]:
from shapely import wkt
parks_detail['isochrones'] = parks_detail['isochrones'].apply(wkt.loads)
parks_detail['centroid'] = parks_detail['centroid'].apply(wkt.loads)
selected_columns = parks_detail[['OBJECTID', 'isochrones','centroid']]
parks_isochrones = gpd.GeoDataFrame(selected_columns, geometry='isochrones')
parks_isochrones.set_crs(epsg=projected_crs,inplace=True)
parks_isochrones = parks_isochrones.to_crs(epsg=projected_crs)
parks_isochrones.head()

,OBJECTID,isochrones,centroid
0,660345,"POLYGON ((-13623319.47 6031677.956, -13623585....",POINT (-13622744.893154 6023634.083662)
1,659362,"POLYGON ((-13612891.839 6039350.079, -13613003...",POINT (-13622815.0938 6023678.225553)
2,659360,"POLYGON ((-13621166.439 6062868.22, -13621376....",POINT (-13622787.819986 6023732.238333)
3,660174,"POLYGON ((-13611805.027 6028358.78, -13611916....",POINT (-13622852.340365 6023754.704958)
4,659359,"POLYGON ((-13616346.083 6038320.133, -13616568...",POINT (-13622926.851761 6023768.188662)


In [6]:
# 加载数据
shapefile_path = 'census_2020.shp'
census_gdf = gpd.read_file(shapefile_path)
gdf_proj = census_gdf.to_crs(epsg=projected_crs)
# 列名映射字典
column_renames = {
    'TOTAL_POPU':'TOTAL_POPULATION',
    'Children_u':'CHILDREN_UNDER_5',
    'Older_Adul':'OLDER_ADULTS_65',
    'LESS_THAN_':'LESS_THAN_HIGH_SCHOOL',
    'BACHELOR_H':'BACHELOR_HIGHER',
    'TOTAL_HO_1':'TOTAL_HOUSING_UNITS',
    'LOW_DENSIT':'LOW_DENSITY',
    'MEDIUM_DEN':'MEDIUM_DENSITY',
    'UNIT_50_MO':'HIGHER_DENSITY',
    'OVERCROWDE':'OVERCROWDED',
    'POP_20_643':'EMPLOYED',
    'PER_CAPITA':'PER_CAPITA_INCOME',
    'NOT_HISPAN': 'WHITE_PEOPLE',
    'NOTHISPLAT': 'BLACK_PEOPLE',
    'NOTHISPL_1': 'INDIAN_ALASKA_PEOPLE',
    'NOTHISPL_2': 'ASIAN_PEOPLE',
    'NOTHISPL_3': 'HAWAIIAN_PACIFIC_PEOPLE',
    'NOTHISPL_4': 'OTHER_RACE_PEOPLE',
    'NOTHISPL_5': 'TWO_OR_MORE_RACES_PEOPLE',
    'HISPANIC_O': 'HISPANIC_PEOPLE',
    'PEOPLE_OF_': 'PEOPLE_OF_COLOR',
    'Median_Age': 'MEDIAN_AGE'
}

# 修改列名
gdf_proj = gdf_proj.rename(columns=column_renames)

# 计算人口密度
gdf_proj['POP_DENSITY'] = gdf_proj['TOTAL_POPULATION']  / (gdf_proj['ACRES_LAND']+ gdf_proj['ACRES_WATE']) * 100
# 计算就业率
gdf_proj['EMPLOYED_PERCENTAGE'] = gdf_proj['EMPLOYED'] / gdf_proj['TOTAL_POPULATION'] * 100
# 计算5岁以下的概率 
gdf_proj['UNDER5_PERCENTAGE'] = gdf_proj['CHILDREN_UNDER_5'] / gdf_proj['TOTAL_POPULATION'] * 100
# 65岁以上
gdf_proj['65_PERCENTAGE'] = gdf_proj['OLDER_ADULTS_65'] / gdf_proj['TOTAL_POPULATION'] * 100
# 有色人种的比例
gdf_proj['PEOPLE_OF_COLOR_PERCENTAGE'] = gdf_proj['PEOPLE_OF_COLOR'] / gdf_proj['TOTAL_POPULATION'] * 100
# 贫穷人口比例
gdf_proj['POVERTY_PERCENTAGE'] = (gdf_proj['INC_POV_RA'] + gdf_proj['INC_POV__1']) / gdf_proj['TOTAL_POPULATION'] * 100

geojson_path = 'SeattleDemographic_Block.geojson'
# 保存为新的 GeoJSON 文件
gdf_proj.to_file(geojson_path, driver='GeoJSON')
print(f"Geojsonfile has been renamed and saved to {geojson_path}")

Geojsonfile has been renamed and saved to SeattleDemographic_Block.geojson


In [9]:
# 使用 'intersects' 或 'within' 进行空间连接
joined = gpd.sjoin(gdf_proj,parks_isochrones, how='inner', predicate='intersects')
joined.head()

,GEOID_20,ACRES_LAND,ACRES_WATE,TRACT,BG,TRBG,BG_NAME,TRACT_LABE,C_DISTRICT,VILLNUMB,...,geometry,POP_DENSITY,EMPLOYED_PERCENTAGE,UNDER5_PERCENTAGE,65_PERCENTAGE,PEOPLE_OF_COLOR_PERCENTAGE,POVERTY_PERCENTAGE,index_right,OBJECTID,centroid
0,530330105022,93.418437,0.0,10502,2,10502.2,Block Group 2,105.02,1,0.0,...,"POLYGON ((-13624397.027 6033490.507, -13624398...",2236.175294,57.683102,4.164672,17.041647,26.089038,9.430349,1955,660623,POINT (-13615460.814991 6048239.588305)
0,530330105022,93.418437,0.0,10502,2,10502.2,Block Group 2,105.02,1,0.0,...,"POLYGON ((-13624397.027 6033490.507, -13624398...",2236.175294,57.683102,4.164672,17.041647,26.089038,9.430349,892,660617,POINT (-13626963.170642 6047300.020903)
0,530330105022,93.418437,0.0,10502,2,10502.2,Block Group 2,105.02,1,0.0,...,"POLYGON ((-13624397.027 6033490.507, -13624398...",2236.175294,57.683102,4.164672,17.041647,26.089038,9.430349,970,659907,POINT (-13611736.217691 6026898.179723)
0,530330105022,93.418437,0.0,10502,2,10502.2,Block Group 2,105.02,1,0.0,...,"POLYGON ((-13624397.027 6033490.507, -13624398...",2236.175294,57.683102,4.164672,17.041647,26.089038,9.430349,1564,660497,POINT (-13616484.301175 6038294.181898)
0,530330105022,93.418437,0.0,10502,2,10502.2,Block Group 2,105.02,1,0.0,...,"POLYGON ((-13624397.027 6033490.507, -13624398...",2236.175294,57.683102,4.164672,17.041647,26.089038,9.430349,1333,659415,POINT (-13613221.055336 6035631.338853)


In [13]:
# 计算每个等时范围内的总人口密度和/或平均人口密度
# 假设 'density' 是人口密度的列名，'OBJECTID' 是等时范围的唯一标识符列名

# 计算平均人口密度
average_density = joined.groupby('OBJECTID').agg({
    'POP_DENSITY':'mean',
    'EMPLOYED_PERCENTAGE':'mean',
    'UNDER5_PERCENTAGE':'mean',
    '65_PERCENTAGE':'mean',
    'PEOPLE_OF_COLOR_PERCENTAGE':'mean',
    'POVERTY_PERCENTAGE':'mean'
}).reset_index()
average_density.head()

,OBJECTID,POP_DENSITY,EMPLOYED_PERCENTAGE,UNDER5_PERCENTAGE,65_PERCENTAGE,PEOPLE_OF_COLOR_PERCENTAGE,POVERTY_PERCENTAGE
0,658736,3810.171223,71.304379,3.498425,8.608949,36.059198,7.691498
1,658737,1155.846725,45.255925,6.406198,14.604505,72.800813,13.735435
2,658738,609.178640,45.584900,3.566884,20.829435,24.834771,2.460945
3,658739,2172.068963,76.271880,2.686745,6.217388,27.411512,7.667199
4,658740,1430.020946,54.140358,3.122537,19.770997,58.161291,6.913684


In [14]:
# 合并结果到等时矩阵数据
parks_isochrones = parks_isochrones.merge(average_density, on='OBJECTID', how='left')
parks_isochrones.head()

,OBJECTID,isochrones,centroid,POP_DENSITY,EMPLOYED_PERCENTAGE,UNDER5_PERCENTAGE,65_PERCENTAGE,PEOPLE_OF_COLOR_PERCENTAGE,POVERTY_PERCENTAGE
0,660345,"POLYGON ((-13623319.47 6031677.956, -13623585....",POINT (-13622744.893154 6023634.083662),1340.473015,53.224039,7.445560,14.533173,28.095383,8.705330
1,659362,"POLYGON ((-13612891.839 6039350.079, -13613003...",POINT (-13622815.0938 6023678.225553),377.397669,50.535796,4.222496,18.855790,22.278383,2.527066
2,659360,"POLYGON ((-13621166.439 6062868.22, -13621376....",POINT (-13622787.819986 6023732.238333),993.086573,41.608194,7.457257,16.950183,27.327034,6.398129
3,660174,"POLYGON ((-13611805.027 6028358.78, -13611916....",POINT (-13622852.340365 6023754.704958),1138.385089,44.467581,6.452216,15.853316,75.931649,15.587020
4,659359,"POLYGON ((-13616346.083 6038320.133, -13616568...",POINT (-13622926.851761 6023768.188662),1248.159756,57.177373,3.115235,17.274209,59.697262,13.623020


In [15]:
parks_isochrones.to_csv('parks_isochrones_with_demographic.csv')